In [2]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
from dask.distributed import Client
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)

import ipywidgets as widgets
from IPython.display import display

In [4]:
# ds = xr.load_dataset("../data/outputs/plots/metrics/metrics_grid_none_z_1m.nc")
ds = xr.load_dataset("../data/outputs/plots/metrics/metrics_grid_none_z_1m_w.nc")


In [5]:
profile_ds = ds[['inside_pct', 'pad', 'ppi']]
profile_ds['pad_pct'] = (profile_ds['pad'] / profile_ds['pad'].sum(dim='z')) * 100
profile_ds['pulse_capture_pct'] = (1 - profile_ds['ppi']) * 100
profile_ds

<xarray.Dataset> Size: 1MB
Dimensions:            (plot: 315, z: 108)
Coordinates:
  * z                  (z) float64 864B 0.0 1.0 2.0 3.0 ... 105.0 106.0 107.0
  * plot               (plot) <U12 15kB 'AGG_O_01_P1' ... 'ULY_Y_96_P5'
    site               (plot) <U9 11kB 'AGG_O_01' 'AGG_O_01' ... 'ULY_Y_96'
    site_type          (plot) <U3 4kB 'AGG' 'AGG' 'AGG' ... 'ULY' 'ULY' 'ULY'
Data variables:
    inside_pct         (plot, z) float64 272kB 23.29 19.19 2.207 ... nan nan nan
    pad                (plot, z) float64 272kB 0.0 0.6009 0.05064 ... nan nan
    ppi                (plot, z) float64 272kB 0.0 0.5483 0.9506 ... nan nan nan
    pad_pct            (plot, z) float64 272kB 0.0 41.24 3.476 ... nan nan nan
    pulse_capture_pct  (plot, z) float64 272kB 100.0 45.17 4.938 ... nan nan nan
Attributes:
    xy_bin_size:  None
    z_bin_size:   1
    weighted:     True

In [6]:
df = profile_ds.to_dataframe().reset_index()
df = df.dropna(subset='inside_pct')
df

,plot,z,inside_pct,pad,ppi,site,site_type,pad_pct,pulse_capture_pct
0,AGG_O_01_P1,0.0,23.293498,0.000000,0.000000,AGG_O_01,AGG,0.000000,100.000000
1,AGG_O_01_P1,1.0,19.187208,0.600876,0.548331,AGG_O_01,AGG,41.240728,45.166876
2,AGG_O_01_P1,2.0,2.206784,0.050644,0.950617,AGG_O_01,AGG,3.475892,4.938259
3,AGG_O_01_P1,3.0,1.939002,0.042475,0.958414,AGG_O_01,AGG,2.915264,4.158584
4,AGG_O_01_P1,4.0,2.462705,0.051470,0.949832,AGG_O_01,AGG,3.532619,5.016796
...,...,...,...,...,...,...,...,...,...
33944,ULY_Y_96_P5,32.0,0.063942,0.000640,0.999360,ULY_Y_96,ULY,0.052187,0.063989
33945,ULY_Y_96_P5,33.0,0.042231,0.000423,0.999578,ULY_Y_96,ULY,0.034449,0.042244
33946,ULY_Y_96_P5,34.0,0.020122,0.000201,0.999799,ULY_Y_96,ULY,0.016409,0.020125
33947,ULY_Y_96_P5,35.0,0.008208,0.000082,0.999918,ULY_Y_96,ULY,0.006692,0.008208


In [7]:
df.to_json("../data/outputs/plots/plot_height_profiles_weighted.json", orient="records")
df.to_parquet("../data/outputs/plots/plot_height_profiles_weighted.parquet", index=False)

<xarray.Dataset> Size: 1MB
Dimensions:            (plot: 315, z: 108)
Coordinates:
  * z                  (z) float64 864B 0.0 1.0 2.0 3.0 ... 105.0 106.0 107.0
  * plot               (plot) <U12 15kB 'AGG_O_01_P1' ... 'ULY_Y_96_P5'
    site               (plot) <U9 11kB 'AGG_O_01' 'AGG_O_01' ... 'ULY_Y_96'
    site_type          (plot) <U3 4kB 'AGG' 'AGG' 'AGG' ... 'ULY' 'ULY' 'ULY'
Data variables:
    inside_pct         (plot, z) float64 272kB 23.29 19.19 2.207 ... nan nan nan
    pad                (plot, z) float64 272kB 0.0 0.6009 0.05064 ... nan nan
    ppi                (plot, z) float64 272kB 0.0 0.5483 0.9506 ... nan nan nan
    pad_pct            (plot, z) float64 272kB 0.0 41.24 3.476 ... nan nan nan
    pulse_capture_pct  (plot, z) float64 272kB 100.0 45.17 4.938 ... nan nan nan
Attributes:
    xy_bin_size:  None
    z_bin_size:   1
    weighted:     True

In [7]:
rel_profile_ds = ds[['qz_inside_pct', 'qz_pad']]
rel_df = rel_profile_ds.to_dataframe().reset_index()
# rel_def = rel_def.dropna(subset=['inside_pct', 'pad'], how='all')
rel_df

,plot,qz,qz_inside_pct,qz_pad,site,site_type
0,AGG_O_01_P1,0.0,21.584704,0.000000,AGG_O_01,AGG
1,AGG_O_01_P1,2.0,14.254737,1.557705,AGG_O_01,AGG
2,AGG_O_01_P1,4.0,2.595799,0.214812,AGG_O_01,AGG
3,AGG_O_01_P1,6.0,1.107567,0.087273,AGG_O_01,AGG
4,AGG_O_01_P1,8.0,0.877665,0.067439,AGG_O_01,AGG
...,...,...,...,...,...,...
16060,ULY_Y_96_P5,92.0,0.061132,0.000867,ULY_Y_96,ULY
16061,ULY_Y_96_P5,94.0,0.039923,0.000566,ULY_Y_96,ULY
16062,ULY_Y_96_P5,96.0,0.022456,0.000318,ULY_Y_96,ULY
16063,ULY_Y_96_P5,98.0,0.013723,0.000195,ULY_Y_96,ULY


In [9]:
rel_df.to_json("../data/outputs/plots/plot_rel_height_profiles.json", orient="records")
rel_df.to_parquet("../data/outputs/plots/plot_rel_height_profiles.parquet", index=False)